In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import List
from collections import defaultdict
import random
from multiprocessing import Process, Queue , Array
import pickle

class RayData:
  def __init__(self, tid, ray_orig_x, ray_orig_y, ray_orig_z, ray_dir_x, ray_dir_y, ray_dir_z):
    self.tid = tid

    self.ray_orig_x=ray_orig_x
    self.ray_orig_y=ray_orig_y
    self.ray_orig_z=ray_orig_z

    self.ray_dir_x=ray_dir_x
    self.ray_dir_y=ray_dir_y
    self.ray_dir_z=ray_dir_z

  def __eq__(self, other):
    if isinstance(other, RayData):
      # Compare attributes for equality
      return  self.tid == other.tid and \
              self.ray_orig_x == other.ray_orig_x and \
              self.ray_orig_y == other.ray_orig_y and \
              self.ray_orig_z == other.ray_orig_z and \
              self.ray_dir_x == other.ray_dir_x and \
              self.ray_dir_y == other.ray_dir_y and \
              self.ray_dir_z == other.ray_dir_z
    return False
  
  def __str__(self):
    # Customize the string representation for printing
    return f"ray_data instance: {self.tid}, {self.ray_orig_x}, {self.ray_orig_y}, {self.ray_orig_z}, {self.ray_dir_x}, {self.ray_dir_y}, {self.ray_dir_z}"

class MemEntry:
  def __init__(self, address, size, type):
    self.entry = [address, size, type]

  def __eq__(self, other):
    if isinstance(other, MemEntry):
      # Compare attributes for equality
      return self.entry[0] == other.entry[0] and self.entry[1] == other.entry[1]
    return False
  
  def __str__(self):
    # Customize the string representation for printing
    return f"mem_entry instance: {self.entry}"

class TraceRayEntry:
  def __init__(self, ray_data: RayData, mem_entries: List[MemEntry]):
    self.ray_data = ray_data
    self.mem_entries = mem_entries

  def __str__(self):
    # Customize the string representation for printing
    return f"trace_ray_entry instance: {self.ray_data}, {self.mem_entries}"

class HashedRay:
  def __init__(self, hash, tid: int, rayid: int):
    self.hash = hash
    self.tid = tid # thread id, index to the thread_list
    self.rayid = rayid # ray id, index to the entry in the thread_list

  def __str__(self):
    return f"HashedRay: hash:{self.hash}, tid:{self.tid}, rayid:{self.rayid}"

# Quantize direction to a sphere - xyz to theta and phi
# `theta_bits` is used for theta, `theta_bits` + 1 is used for phi, for a total of
# 2 * `theta_bits` + 1 bits
def hash_direction_spherical(d, num_sphere_bits):
  theta_bits = np.uint32(num_sphere_bits)
  phi_bits = np.uint32(theta_bits + 1)

  theta = np.uint64(np.arccos(np.clip(d[2], -1.0, 1.0)) / np.pi * 180)
  phi = np.uint64((np.arctan2(d[1], d[0]) + np.pi) / np.pi * 180)
  q_theta = theta >> np.uint64(8 - theta_bits)
  q_phi = phi >> np.uint64(9 - phi_bits)

  return (q_phi << theta_bits) | q_theta

def hash_origin_grid(o, min_val, max_val, num_bits):
  grid_size = 1 << num_bits

  hash_o_x = np.uint64(np.clip((o[0] - min_val[0]) / (max_val[0] - min_val[0]) * grid_size, 0.0, float(grid_size) - 1))
  hash_o_y = np.uint64(np.clip((o[1] - min_val[1]) / (max_val[1] - min_val[1]) * grid_size, 0.0, float(grid_size) - 1))
  hash_o_z = np.uint64(np.clip((o[2] - min_val[2]) / (max_val[2] - min_val[2]) * grid_size, 0.0, float(grid_size) - 1))
  
  hash_value = (hash_o_x << np.uint32((2 * num_bits))) | (hash_o_y << np.uint32(num_bits)) | hash_o_z
  return np.uint64(hash_value)

def hash_grid_spherical(ray_direction, ray_origin, min_val, max_val, num_grid_bits, num_sphere_bits):
  hash_d = hash_direction_spherical(ray_direction, num_sphere_bits)
  hash_o = hash_origin_grid(ray_origin, min_val, max_val, num_grid_bits)
  hash_value = hash_o ^ hash_d

  return hash_value

def parse_csv(csv, number_of_threads):
  thread_list = [[] for _ in range(number_of_threads)]
  mem_entries = []
  last_idx = csv[0][Indices.IDX]
  last_tid = csv[0][Indices.TID]
  last_ray_data = RayData(csv[0][Indices.TID],
                           csv[0][Indices.RAY_ORIG_X],
                           csv[0][Indices.RAY_ORIG_Y],
                           csv[0][Indices.RAY_ORIG_Z],
                           csv[0][Indices.RAY_DIR_X],
                           csv[0][Indices.RAY_DIR_Y],
                           csv[0][Indices.RAY_DIR_Z])
  print("Parsing the csv...")
  for entry in csv:
    if(last_idx != entry[Indices.IDX]):
      thread_list[last_tid].append(TraceRayEntry(last_ray_data,mem_entries[:]))
      mem_entries.clear()
      last_ray_data = RayData(entry[Indices.TID],
                               entry[Indices.RAY_ORIG_X],
                               entry[Indices.RAY_ORIG_Y],
                               entry[Indices.RAY_ORIG_Z],
                               entry[Indices.RAY_DIR_X],
                               entry[Indices.RAY_DIR_Y],
                               entry[Indices.RAY_DIR_Z])
      last_idx = entry[Indices.IDX]
      last_tid = entry[Indices.TID]
    
    mem_entries.append(MemEntry(int(entry[Indices.ADDR],16),entry[Indices.SIZE],entry[Indices.TYPE]))
  print("Finished parsing the csv...")
  return thread_list

class Indices:
  IDX = 0
  TID = 1
  ADDR = 2
  SIZE = 3
  TYPE = 4
  RAY_ORIG_X = 5
  RAY_ORIG_Y = 6
  RAY_ORIG_Z = 7
  RAY_DIR_X = 8
  RAY_DIR_Y = 9
  RAY_DIR_Z = 10

In [30]:

def load_scene_stereo(scene: str):
  assert(scene == 'sponza' or scene == 'bunny' or scene == 'bunny_ao' or scene == 'sponza_ao')
  print(f"Loading {scene}")
  # Read the first CSV file into a DataFrame
  df1 = pd.read_csv(f'../../outputs/{scene}_left_eye_mem_access.csv')
  # Convert the DataFrame to a list of lists
  csv1 = df1.values.tolist()
  number_of_threads1 = df1['tid'].nunique()

  thread_list1 = parse_csv(csv1,number_of_threads1)

  # Read the second CSV file into a DataFrame
  df2 = pd.read_csv(f'../../outputs/{scene}_right_eye_mem_access.csv')
  # Convert the DataFrame to a list of lists
  csv2 = df2.values.tolist()
  number_of_threads2 = df2['tid'].nunique()

  thread_list2 = parse_csv(csv2,number_of_threads2)

  if scene == 'bunny' or scene == 'bunny_ao':
    min_val = np.array([0.0, 0.0, -555.0])  # bunny min values
    max_val = np.array([556.0, 556.0, 1.0])  # bunny max values
  elif scene == 'sponza' or scene == 'sponza_ao':
    min_val = np.array([-1105.42603,-126.442497,-1920.94592]) # sponza min values
    max_val = np.array([1198.57397,1433.5575,1807.05408]) # sponza max values

  return thread_list1,thread_list2,min_val,max_val

def hash_rays(thread_list1, min_val, max_val, num_grid_bits, num_sphere_bits):
  hash_bits = max(3*num_grid_bits,2*num_sphere_bits+1)
  print(f"Hashing rays...({hash_bits} bits)")
  hash_dict = {}
  for i in range(2**hash_bits):
    hash_dict[i] = []

  for tid,thread in enumerate(thread_list1):
    for rayid,ray in enumerate(thread):
      # only hash if the ray intersects the scene
      if(any(obj.entry[2] == 5 for obj in ray.mem_entries)):
        hash_value = hash_grid_spherical(np.array([ray.ray_data.ray_dir_x, ray.ray_data.ray_dir_y, ray.ray_data.ray_dir_z]),
                                         np.array([ray.ray_data.ray_orig_x, ray.ray_data.ray_orig_y, ray.ray_data.ray_orig_z]),
                                         min_val, max_val,num_grid_bits,num_sphere_bits)
        hash_dict[hash_value].append(HashedRay(hash_value, tid, rayid))

  return hash_dict

def hash_rays_random(thread_list1, num_grid_bits, num_sphere_bits):
  hash_bits = max(3*num_grid_bits,2*num_sphere_bits+1)
  print(f"Hashing rays randomly...({hash_bits} bits)")
  hash_dict = {}
  for i in range(2**hash_bits):
    hash_dict[i] = []

  for tid,thread in enumerate(thread_list1):
    for rayid,ray in enumerate(thread):
      # only hash if the ray intersects the scene
      if(any(obj.entry[2] == 5 for obj in ray.mem_entries)):
        hash_value = random.randint(0, 2**hash_bits-1)
        hash_dict[hash_value].append(HashedRay(hash_value, tid, rayid))

  return hash_dict

def check_matches(list1: List[MemEntry], list2: List[MemEntry]):
  num_matches = 0
  length = len(list2) if len(list2) < len(list1) else len(list1)
  for idx in range(length):
      if(list1[idx].entry[0] == list2[idx].entry[0] and list1[idx].entry[1] == list2[idx].entry[1]):
        num_matches += 1
      else:
        break

  return num_matches

def is_hit(thread_list,tid,rayid):
  for entry in thread_list[tid][rayid].mem_entries:
    if entry.entry[2] == 5:
      return True
  return False

def get_kth_ancestor(thread_list, tid: int, rayid: int, node_parent_map, k: int):
  # Returns the kth ancestor of the closest hit of the ray
  last_hit_node_addr = 0
  for idx,entry in enumerate(thread_list[tid][rayid].mem_entries):
    if entry.entry[2] == 5:
      last_hit_node_addr = entry.entry[0]
  if(last_hit_node_addr == 0):
    return 0
  curr_node = last_hit_node_addr
  for i in range(k):
    curr_node = node_parent_map[curr_node]
    if(curr_node == 0):
      return 0
  return curr_node

def find_all_matches(hash_dict1, hash_dict2, thread_list1, thread_list2, node_parent_map, k: int=3):
  matches_list = []
  for hash2,hash_list2 in hash_dict2.items():
    num_matching_hashes = max(len(hash_dict1[hash2]), 1)
    #num_matching_hashes = max(sum(1 for obj in hash_dict1[hash2] if obj.rayid==0), 1)
    for ray2 in hash_list2:
    #  if(ray2.rayid != 0):
    #    continue
      total_matches = 0
      for ray1 in hash_dict1[hash2]:
    #    if(ray1.rayid != 0):
    #      continue
        if get_kth_ancestor(thread_list1,ray1.tid,ray1.rayid,node_parent_map,k) == get_kth_ancestor(thread_list2,ray2.tid,ray2.rayid,node_parent_map,k):
          total_matches += 1
      matches_list.append(total_matches/num_matching_hashes)

  return np.array(matches_list)

def find_nearest_matches(hash_dict1, hash_dict2, thread_list1, thread_list2):
  matches_list = []
  lengths_list = []
  tid_list = []
  rayid_list = []
  for hash2,hash_list2 in hash_dict2.items():
    nearest_matches = 0
    nearest_tid = 0
    nearest_rayid = 0
    for ray2 in hash_list2:
      min_distance = 32768
      nearest_len = len(thread_list2[ray2.tid][ray2.rayid].mem_entries)
      for ray1 in hash_dict1[hash2]:
        if abs(ray1.tid-ray2.tid) < min_distance:
          min_distance = abs(ray1.tid-ray2.tid)
          matches = check_matches(thread_list1[ray1.tid][ray1.rayid].mem_entries, thread_list2[ray2.tid][ray2.rayid].mem_entries)
          nearest_matches = matches
          nearest_tid = ray1.tid
          nearest_rayid = ray1.rayid
      matches_list.append(nearest_matches)
      lengths_list.append(nearest_len)
      tid_list.append(nearest_tid)
      rayid_list.append(nearest_rayid)

  return np.array(matches_list),np.array(lengths_list),np.array(tid_list),np.array(rayid_list)   

def find_tile16x16_matches(hash_dict1, hash_dict2, thread_list1, thread_list2):
  matches_list = []
  lengths_list = []
  tid_list = []
  rayid_list = []
  count = 0
  for hash2,hash_list2 in hash_dict2.items():
    for ray2 in hash_list2:
      # count += 1
      # if count > 10:
      #   return np.array(matches_list),np.array(lengths_list),np.array(tid_list),np.array(rayid_list)
      total_matches = 0
      num_matching_hashes = 0
      ray2x = ray2.tid%128
      ray2y = ray2.tid//128
      tilex = ray2x-ray2x%16
      tiley = ray2y-ray2y%16
      #print(f"ray2: {ray2x},{ray2y},{tilex},{tiley}")
      ray2_len = len(thread_list2[ray2.tid][ray2.rayid].mem_entries)
      for ray1 in hash_dict1[hash2]:
        ray1x = ray1.tid%128
        ray1y = ray1.tid//128
        #print(f"ray1: {ray1x},{ray1y}")
        if (ray1x-tilex >= 0 and ray1x-tilex < 16 and ray1y-tiley >= 0 and ray1y-tiley < 16):
          matches = check_matches(thread_list1[ray1.tid][ray1.rayid].mem_entries, thread_list2[ray2.tid][ray2.rayid].mem_entries)
          total_matches += matches
          num_matching_hashes += 1
      num_matching_hashes = num_matching_hashes if num_matching_hashes > 0 else 1
      matches_list.append(np.int32(np.ceil(total_matches/num_matching_hashes)))
      lengths_list.append(ray2_len)

  return np.array(matches_list),np.array(lengths_list),np.array(tid_list),np.array(rayid_list)   

# This is for comparing primary rays
def find_all_matches_unhashed(thread_list1, thread_list2, node_parent_map, k:int = 3):
  start = 2000
  end = 3000
  match_list = [[] for _ in range(end-start)]
  tid2 = start
  for thread2 in thread_list2[start:end]:
    if is_hit(thread_list2,tid2,0) == False:
      tid2 += 1
      continue
    ray2_ancestor = get_kth_ancestor(thread_list2,tid2,0,node_parent_map,k)
    #tid1 = max(0,tid2-1000)
    tid1 = 0
    #for thread1 in thread_list1[max(0,tid2-1000):min(tid2+1000,len(thread_list2))]:
    for thread1 in thread_list1:
      if is_hit(thread_list1,tid1,0) == False:
        tid1 += 1
        continue
      ray1_ancestor = get_kth_ancestor(thread_list1,tid1,0,node_parent_map,k)
      if ray2_ancestor == ray1_ancestor:
        #match_list[tid2].append(tid1)
        print(f"t1x,t1y:{tid1%128},{tid1//128},t2x,t2y:{tid2%128},{tid2//128}")
      tid1 += 1
    tid2 += 1

  return match_list

def dump_scene_threadlist_pickle(thread_list1,thread_list2,scene: str):
  with open(f'{scene}_left_threadlist.pickle', 'wb') as f:
    pickle.dump(thread_list1, f)

  with open(f'{scene}_right_threadlist.pickle', 'wb') as f:
    pickle.dump(thread_list2, f)

def dump_scene_hashlist_pickle(hash_list1,hash_list2,scene: str,hash_label: str):
  with open(f'{scene}_left_{hash_label}_hashlist.pickle', 'wb') as f:
    pickle.dump(hash_list1, f)

  with open(f'{scene}_right_{hash_label}_hashlist.pickle', 'wb') as f:
    pickle.dump(hash_list2, f)

def load_scene_threadlist_pickle(scene: str):
  with open(f'{scene}_left_threadlist.pickle', 'rb') as f:
    thread_list1 = pickle.load(f)

  with open(f'{scene}_right_threadlist.pickle', 'rb') as f:
    thread_list2 = pickle.load(f)

  return thread_list1, thread_list2

def load_scene_hashlist_pickle(scene: str, hash_label: str):
  with open(f'{scene}_left_{hash_label}_hashlist.pickle', 'rb') as f:
    hash_list1 = pickle.load(f)

  with open(f'{scene}_right_{hash_label}_hashlist.pickle', 'rb') as f:
    hash_list2 = pickle.load(f)

  return hash_list1,hash_list2

def load_node_parent_map_stereo(scene: str):
  left_filename = "../../outputs/"+scene+"_left_eye_node_parent_map.txt"
  right_filename = "../../outputs/"+scene+"_right_eye_node_parent_map.txt"
  ao_left_filename = "../../outputs/"+scene+"_ao_right_eye_node_parent_map.txt"
  ao_right_filename = "../../outputs/"+scene+"_ao_right_eye_node_parent_map.txt"
  node_parent_map = {}
  with open(left_filename, "r") as f:
    for line in f:
      key, value = line.strip().split(",")
      node_parent_map[int("0x" + key,16)] = int("0x" + value,16)

  with open(right_filename, "r") as f:
    for line in f:
      key, value = line.strip().split(",")
      if int("0x" + key,16) in node_parent_map:
        assert(node_parent_map[int("0x" + key,16)] == int("0x" + value,16))
      else:
        node_parent_map[int("0x" + key,16)] = int("0x" + value,16)

  with open(ao_left_filename, "r") as f:
    for line in f:
      key, value = line.strip().split(",")
      if int("0x" + key,16) in node_parent_map:
        assert(node_parent_map[int("0x" + key,16)] == int("0x" + value,16))
      else:
        node_parent_map[int("0x" + key,16)] = int("0x" + value,16)

  with open(ao_right_filename, "r") as f:
    for line in f:
      key, value = line.strip().split(",")
      if int("0x" + key,16) in node_parent_map:
        assert(node_parent_map[int("0x" + key,16)] == int("0x" + value,16))
      else:
        node_parent_map[int("0x" + key,16)] = int("0x" + value,16)

  return node_parent_map

In [25]:
thread_list1,thread_list2,min_val,max_val =  load_scene_stereo('sponza_ao')

Loading sponza_ao
Parsing the csv...
Finished parsing the csv...
Parsing the csv...
Finished parsing the csv...


In [26]:
hash_dict1 = hash_rays(thread_list1,min_val,max_val,5,3)
hash_dict2 = hash_rays(thread_list2,min_val,max_val,5,3)

Hashing rays...(15 bits)
Hashing rays...(15 bits)


In [ ]:
hash_dict_random1 = hash_rays_random(thread_list1,5,3)
hash_dict_random2 = hash_rays_random(thread_list2,5,3)

In [ ]:
print(thread_list1[5000][0].ray_data)
print(thread_list2[5000][0].ray_data)
matches_list = []
length_list = []
for idx,_ in enumerate(thread_list1):
  if(any(obj.entry[2] == 5 for obj in thread_list2[idx][0].mem_entries)):
    matches = check_matches(thread_list1[idx][0].mem_entries,thread_list2[idx][0].mem_entries)
    length = len(thread_list2[idx][0].mem_entries)
    matches_list.append(matches)
    length_list.append(length)

z = np.array(np.array(matches_list)/np.array(length_list))
np.mean(z)


In [31]:

sponza_node_parent_map = load_node_parent_map_stereo("sponza")
#m_all = find_all_matches(hash_dict1, hash_dict2, thread_list1, thread_list2,bunny_node_parent_map,3)
m_all = find_all_matches_unhashed(thread_list1,thread_list2,sponza_node_parent_map,3)
# for hash2,hash_list2 in hash_dict2.items():
#   for ray2 in hash_list2:
#     if ray2.rayid == 0:
#       print(f"hash: {hash2}, Matching hashes:{len(hash_dict1[hash2])}")
#       print(f"ray2 data: {thread_list2[ray2.tid][ray2.rayid].ray_data}")
#       for ray1 in hash_dict1[hash2]:
#         print(ray1.rayid)
#         print(f"ray1 data: {thread_list2[ray1.tid][ray1.rayid].ray_data}")



t1x,t1y:100,3,t2x,t2y:80,15
t1x,t1y:100,4,t2x,t2y:80,15
t1x,t1y:99,5,t2x,t2y:80,15
t1x,t1y:100,5,t2x,t2y:80,15
t1x,t1y:99,6,t2x,t2y:80,15
t1x,t1y:100,6,t2x,t2y:80,15
t1x,t1y:99,7,t2x,t2y:80,15
t1x,t1y:71,11,t2x,t2y:80,15
t1x,t1y:72,11,t2x,t2y:80,15
t1x,t1y:73,11,t2x,t2y:80,15
t1x,t1y:74,11,t2x,t2y:80,15
t1x,t1y:75,11,t2x,t2y:80,15
t1x,t1y:76,11,t2x,t2y:80,15
t1x,t1y:77,11,t2x,t2y:80,15
t1x,t1y:78,11,t2x,t2y:80,15
t1x,t1y:79,11,t2x,t2y:80,15
t1x,t1y:80,11,t2x,t2y:80,15
t1x,t1y:81,11,t2x,t2y:80,15
t1x,t1y:82,11,t2x,t2y:80,15
t1x,t1y:83,11,t2x,t2y:80,15
t1x,t1y:84,11,t2x,t2y:80,15
t1x,t1y:72,12,t2x,t2y:80,15
t1x,t1y:73,12,t2x,t2y:80,15
t1x,t1y:74,12,t2x,t2y:80,15
t1x,t1y:75,12,t2x,t2y:80,15
t1x,t1y:76,12,t2x,t2y:80,15
t1x,t1y:77,12,t2x,t2y:80,15
t1x,t1y:78,12,t2x,t2y:80,15
t1x,t1y:79,12,t2x,t2y:80,15
t1x,t1y:80,12,t2x,t2y:80,15
t1x,t1y:81,12,t2x,t2y:80,15
t1x,t1y:82,12,t2x,t2y:80,15
t1x,t1y:83,12,t2x,t2y:80,15
t1x,t1y:84,12,t2x,t2y:80,15
t1x,t1y:73,13,t2x,t2y:80,15
t1x,t1y:74,13,t2x,t2y:8

In [37]:
import matplotlib.pyplot as plt

# Read data from the file
data_file = "data.txt"

# Dictionary to store data grouped by t2x, t2y coordinates
data_by_t2 = {}

# Read data from the file and group it by t2x, t2y coordinates
with open(data_file, 'r') as file:      
  for line_num,line in enumerate(file):
    if line_num < 63588:
      continue
    line = line.strip()
    if line:
      parts = line.split(',')
      t2_coords = tuple(map(int, [parts[4].split(':')[1],parts[5]]))
      t1_coords = tuple(map(int, [parts[1].split(':')[1],parts[2]]))
      if t2_coords not in data_by_t2:
        data_by_t2[t2_coords] = []
      data_by_t2[t2_coords].append(t1_coords)
      if len(data_by_t2) > 10:
        break

# Generate plots for each unique t2x, t2y coordinate
for t2_coords, t1_coords_list in data_by_t2.items():
  plt.figure(figsize=(4, 4))  # Adjust figure size as needed
  plt.title(f"t2x, t2y: {t2_coords}")

  # Plot t1x, t1y coordinates
  for t1_coords in t1_coords_list:
      plt.scatter(t1_coords[0], t1_coords[1], color='red')

  plt.scatter(t2_coords[0], t2_coords[1], color='blue', label='t2 coords')

  plt.xlim(0, 128)
  plt.ylim(0, 128)
  plt.xlabel('t1x')
  plt.ylabel('t1y')
  plt.grid(True)
  plt.gca().invert_yaxis()
  plt.gca().set_aspect('equal', adjustable='box')
  plt.savefig(f"plot_t2_{t2_coords[0]}_{t2_coords[1]}.png")
  plt.close()
